In [1]:
import glob
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import json
import numpy as np
import scipy.stats
from loaddata import *
import collections
from mapping import *
import tqdm
from loaddata import *
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [723]:
class_map = ClassMapping()

In [724]:
sub_tmp = pd.read_csv("sample_submission.csv")

In [1048]:
df_test_info_gps = load_testinfo_gps()
df_test_info_gps.head()

,height,image_id,seq_id,location,width,datetime,file_name,seq_frame_num,seq_num_frames,sub_location,CC
0,1024,8b31d3be-21bc-11ea-a13a-137349068a90,a91ebc18-0cd3-11eb-bed1-0242ac1c0002,20,1280,2013-06-09 16:01:38.000,8b31d3be-21bc-11ea-a13a-137349068a90.jpg,0,10,NaN,BZ
1,1024,8cf202be-21bc-11ea-a13a-137349068a90,a91ebc18-0cd3-11eb-bed1-0242ac1c0002,20,1280,2013-06-09 16:01:39.000,8cf202be-21bc-11ea-a13a-137349068a90.jpg,1,10,NaN,BZ
2,1024,8a87e62e-21bc-11ea-a13a-137349068a90,a91ebc18-0cd3-11eb-bed1-0242ac1c0002,20,1280,2013-06-09 16:01:40.000,8a87e62e-21bc-11ea-a13a-137349068a90.jpg,2,10,NaN,BZ
3,1024,8e6994f4-21bc-11ea-a13a-137349068a90,a91ebc18-0cd3-11eb-bed1-0242ac1c0002,20,1280,2013-06-09 16:01:41.000,8e6994f4-21bc-11ea-a13a-137349068a90.jpg,3,10,NaN,BZ
4,1024,948b29e2-21bc-11ea-a13a-137349068a90,a91ebc18-0cd3-11eb-bed1-0242ac1c0002,20,1280,2013-06-09 16:01:42.000,948b29e2-21bc-11ea-a13a-137349068a90.jpg,4,10,NaN,BZ


In [725]:
sub_tmp.head()

,Id,Predicted2,Predicted3,Predicted4,Predicted6,Predicted7,Predicted8,Predicted9,Predicted10,Predicted12,...,Predicted559,Predicted562,Predicted563,Predicted564,Predicted565,Predicted566,Predicted567,Predicted568,Predicted570,Predicted571
0,32ce8026-7ec9-11eb-b675-4f3cc0c82eb3,4,2,8,3,6,6,6,3,6,...,6,4,1,0,9,5,5,1,5,8
1,945c6602-21bc-11ea-a13a-137349068a90,2,5,5,8,5,4,2,3,1,...,0,8,4,3,9,1,6,7,2,4
2,a91c7e26-0cd3-11eb-bed1-0242ac1c0002,0,0,3,5,8,3,5,4,6,...,2,3,9,6,3,7,9,2,6,5
3,9926239e-21bc-11ea-a13a-137349068a90,6,9,4,5,1,5,3,1,9,...,6,0,5,1,9,0,4,9,7,7
4,9672184c-21bc-11ea-a13a-137349068a90,4,1,5,8,8,4,1,7,1,...,2,4,3,2,0,6,5,3,9,3


In [726]:
sub_tmp.sum()

Id              32ce8026-7ec9-11eb-b675-4f3cc0c82eb3945c6602-2...
Predicted2                                                  49741
Predicted3                                                  49739
Predicted4                                                  49647
Predicted6                                                  49096
                                      ...                        
Predicted566                                                49810
Predicted567                                                49595
Predicted568                                                49721
Predicted570                                                50399
Predicted571                                                50257
Length: 205, dtype: object

In [3]:
#results = loadmergedresults(["20210428","20210428","20210428","20210426","20210503BZ","20210503ZZ","20210503KE"])
#results = loadmergedresults(["20210521"])
results = loadmergedresults(["20210428"])

Dir 20210428
Read  20210428/topk_ids1619596859.256612.csv 33435
Read  20210428/topk_ids1619596859.256612.csv 33435
Read  20210428/topk_ids1619597745.112246.csv 33435
Read  20210428/topk_ids1619595413.719021.csv 33435
Read  20210428/topk_ids1619595950.469775.csv 33435
Read  20210428/topk_ids1619596318.508688.csv 33435
Read  20210428/topk_ids1619597036.607354.csv 33435
Read  20210428/topk_ids1619594881.455932.csv 33435
Read  20210428/topk_ids1619594703.184153.csv 33435
Read  20210428/topk_ids1619598102.916984.csv 33435
Read  20210428/topk_ids1619598459.33737.csv 33435
Read  20210428/topk_ids1619595771.425337.csv 33435
Read  20210428/topk_ids1619597568.122965.csv 33435
Read  20210428/topk_ids1619598280.762844.csv 33435
Read  20210428/topk_ids1619594348.655666.csv 33435
Read  20210428/topk_ids1619596504.704484.csv 33435
Read  20210428/topk_ids1619599528.380561.csv 33435
Read  20210428/topk_ids1619599168.106336.csv 33435
Read  20210428/topk_ids1619598991.020862.csv 33435
Read  20210428/topk

In [4]:
print(len(results))
print(results[results.isnull().sum(axis=1) > 0])
results.head()

33435
Empty DataFrame
Columns: [filename, net_id, confidence, image_id, idx]
Index: []


,filename,net_id,confidence,image_id,idx
0,8a0a5ea2-21bc-11ea-a13a-137349068a90_1.jpg,169,0.717121,8a0a5ea2-21bc-11ea-a13a-137349068a90,1
1,8a0a30c6-21bc-11ea-a13a-137349068a90_1.jpg,24,0.853304,8a0a30c6-21bc-11ea-a13a-137349068a90,1
2,8a0a30c6-21bc-11ea-a13a-137349068a90_2.jpg,24,0.882559,8a0a30c6-21bc-11ea-a13a-137349068a90,2
3,8a0a92a0-21bc-11ea-a13a-137349068a90_1.jpg,164,0.913180,8a0a92a0-21bc-11ea-a13a-137349068a90,1
4,8a0a4156-21bc-11ea-a13a-137349068a90_1.jpg,98,0.690781,8a0a4156-21bc-11ea-a13a-137349068a90,1


In [5]:
sub = pd.read_csv("sample_submission.csv")
col_Predicted = [col for col in sub.columns if "Predicted" in col]

In [6]:
# count all categories per image
pred = collections.defaultdict(list)
empty = collections.defaultdict(list)

for i, row in results.iterrows():
    image_id = row["image_id"]
    category = int(row["net_id"])
    confidence = float(row["confidence"])
    if confidence >= 0.79:
        pred[image_id].append(category)
    else:
        empty[image_id].append(category)

In [10]:
duplicates = (set(empty) & set(pred))

In [16]:
for d in duplicates:
    print(d,empty[d],pred[d])
    for e in empty[d]:
        pred[d].append(pred[d][0])
    print(d,empty[d],pred[d])
        

928ccb00-21bc-11ea-a13a-137349068a90 [38] [38, 38]
928ccb00-21bc-11ea-a13a-137349068a90 [38] [38, 38, 38]
88da4b50-21bc-11ea-a13a-137349068a90 [54] [54, 54]
88da4b50-21bc-11ea-a13a-137349068a90 [54] [54, 54, 54]
8f4b077c-21bc-11ea-a13a-137349068a90 [71] [71, 71]
8f4b077c-21bc-11ea-a13a-137349068a90 [71] [71, 71, 71]
8691c634-21bc-11ea-a13a-137349068a90 [54] [24, 24]
8691c634-21bc-11ea-a13a-137349068a90 [54] [24, 24, 24]
91806c3a-21bc-11ea-a13a-137349068a90 [54] [54, 54]
91806c3a-21bc-11ea-a13a-137349068a90 [54] [54, 54, 54]
8fae0534-21bc-11ea-a13a-137349068a90 [24] [24, 24, 24]
8fae0534-21bc-11ea-a13a-137349068a90 [24] [24, 24, 24, 24]
8dc4584a-21bc-11ea-a13a-137349068a90 [29] [38, 38]
8dc4584a-21bc-11ea-a13a-137349068a90 [29] [38, 38, 38]
87b8ac08-21bc-11ea-a13a-137349068a90 [4] [4, 4]
87b8ac08-21bc-11ea-a13a-137349068a90 [4] [4, 4, 4]
90d4314a-21bc-11ea-a13a-137349068a90 [162] [162, 162]
90d4314a-21bc-11ea-a13a-137349068a90 [162] [162, 162, 162]
8c3fb776-21bc-11ea-a13a-137349068a90 [

8add78dc-21bc-11ea-a13a-137349068a90 [163] [54, 54, 54]
88309308-21bc-11ea-a13a-137349068a90 [17] [164, 164, 164]
88309308-21bc-11ea-a13a-137349068a90 [17] [164, 164, 164, 164]
89a49266-21bc-11ea-a13a-137349068a90 [34] [34, 34, 34]
89a49266-21bc-11ea-a13a-137349068a90 [34] [34, 34, 34, 34]
97518ad6-21bc-11ea-a13a-137349068a90 [163] [163, 163]
97518ad6-21bc-11ea-a13a-137349068a90 [163] [163, 163, 163]
9085bc72-21bc-11ea-a13a-137349068a90 [2] [2, 2]
9085bc72-21bc-11ea-a13a-137349068a90 [2] [2, 2, 2]
88d959ca-21bc-11ea-a13a-137349068a90 [17] [24, 24]
88d959ca-21bc-11ea-a13a-137349068a90 [17] [24, 24, 24]
926fff02-21bc-11ea-a13a-137349068a90 [106] [106, 106, 106, 106, 106, 106, 106]
926fff02-21bc-11ea-a13a-137349068a90 [106] [106, 106, 106, 106, 106, 106, 106, 106]
89022634-21bc-11ea-a13a-137349068a90 [4] [4, 4]
89022634-21bc-11ea-a13a-137349068a90 [4] [4, 4, 4]
92b604d4-21bc-11ea-a13a-137349068a90 [38] [38, 38]
92b604d4-21bc-11ea-a13a-137349068a90 [38] [38, 38, 38]
8f897200-21bc-11ea-a13a

In [1058]:
print(len(pred))
# 0.0     25985
# 0.8     15637

# 0.799   15689
# 0.79    4746
# 0.75    17800
# 0.7     19541


4727


In [1017]:
# we already work on modes, cattle is cattle so probably the same animals is in the group
#for im_id in pred.keys():
#    v = scipy.stats.mode([p for p in pred[im_id]])[0][0]
#    pred[im_id] = [v] * len(pred[im_id])

In [1018]:
for p in pred:
    if len(set(pred[p])) > 1:
        print(p,set(pred[p]))

8c3b5d3e-21bc-11ea-a13a-137349068a90 {106, 38}
8e9dbe00-21bc-11ea-a13a-137349068a90 {54, 38}
8e21e596-21bc-11ea-a13a-137349068a90 {106, 38}
88a10034-21bc-11ea-a13a-137349068a90 {106, 38}
91cd6d0a-21bc-11ea-a13a-137349068a90 {106, 38}
93bf8e4a-21bc-11ea-a13a-137349068a90 {106, 38}
95fb6940-21bc-11ea-a13a-137349068a90 {106, 38}
975d88f4-21bc-11ea-a13a-137349068a90 {106, 38}
9387a304-21bc-11ea-a13a-137349068a90 {106, 38}
93648ee6-21bc-11ea-a13a-137349068a90 {106, 38}
96883ed8-21bc-11ea-a13a-137349068a90 {54, 38}
90088464-21bc-11ea-a13a-137349068a90 {106, 38}


In [1019]:
# make a list of counters that matches the submission format already
counters = []

for image_id in pred.keys():
    c = collections.Counter(pred[image_id])
    
    res = []
    cnts = [0 for i in range(205)]
    for category, cnt in c.items():
        cnts[category] = cnt
    res += [image_id] + cnts[1:]
    counters.append(res)

In [1020]:
merged = load_merged()

In [1021]:
sequences = merged["seq_id"].unique()

In [1022]:
df_test_info = load_testinfo()
sequences = df_test_info["seq_id"].unique()
print(len(df_test_info))
len(sequences)

60214


11057

In [1023]:
merged.head()

,id,image_id,category_id,name,seq_num_frames,location,datetime,seq_id,width,height,file_name,sub_location,seq_frame_num,detections,max_detection_conf,nr_boxes
0,a292dd3c-21bc-11ea-a13a-137349068a90,96b00332-21bc-11ea-a13a-137349068a90,73,canis lupus,1,267,2013-08-08 11:45:00.000,96b004ea-21bc-11ea-a13a-137349068a90,1795,1222,96b00332-21bc-11ea-a13a-137349068a90.jpg,NaN,0,"[{'category': '1', 'bbox': [0.4005, 0.6587, 0....",0.995,1
1,9b075d72-21bc-11ea-a13a-137349068a90,973073b4-21bc-11ea-a13a-137349068a90,73,canis lupus,3,521,2014-06-23 17:16:00.000,97307576-21bc-11ea-a13a-137349068a90,2048,1536,973073b4-21bc-11ea-a13a-137349068a90.jpg,NaN,0,"[{'category': '1', 'bbox': [0.4144, 0.1232, 0....",1.000,1
2,a0d6f7b2-21bc-11ea-a13a-137349068a90,89588844-21bc-11ea-a13a-137349068a90,73,canis lupus,3,521,2014-06-23 17:16:00.000,97307576-21bc-11ea-a13a-137349068a90,2048,1536,89588844-21bc-11ea-a13a-137349068a90.jpg,NaN,1,"[{'category': '1', 'bbox': [0.1791, 0.0985, 0....",1.000,1
3,a2097290-21bc-11ea-a13a-137349068a90,87dc5806-21bc-11ea-a13a-137349068a90,73,canis lupus,3,521,2014-06-23 17:16:01.000,97307576-21bc-11ea-a13a-137349068a90,2048,1536,87dc5806-21bc-11ea-a13a-137349068a90.jpg,NaN,2,"[{'category': '1', 'bbox': [0.0439, 0.0355, 0....",1.000,1
4,9a4c7b1a-21bc-11ea-a13a-137349068a90,96960680-21bc-11ea-a13a-137349068a90,73,canis lupus,3,140,2015-02-12 10:10:05.000,974c0124-21bc-11ea-a13a-137349068a90,1280,720,96960680-21bc-11ea-a13a-137349068a90.jpg,NaN,0,"[{'category': '1', 'bbox': [0.3808, 0.4022, 0....",1.000,2


In [208]:
def calc2(image_ids):
    lengths = []
    vals = []
    for im_id in image_ids:
        im_id = str(im_id)
        lengths.append(len(pred[im_id]))
        vals.extend(pred[im_id])
#        print(pred[im_id])
    if len(vals) > 0:
        v = scipy.stats.mode(vals)[0][0]
    else:
        v = 0
    return v, np.max(lengths)

In [209]:
# make a list of counters that matches the submission format already
counters = []

for i, seq in enumerate(tqdm.tqdm(sequences)):
    images = df_test_info[df_test_info["seq_id"] == seq]
#    print(images["image_id"])
    image_ids = [str(im["image_id"]) for i,im in images.iterrows()]
    category, count = calc2(image_ids)
    c = collections.Counter(seq)
    res = []
    cnts = [0 for i in range(205)]
    
    if(category > 0):
        cnts[category] = count
        
    res += [seq] + cnts[1:]
    counters.append(res)

100%|██████████| 11057/11057 [00:32<00:00, 337.15it/s]


In [1024]:
new_tmp = pd.DataFrame(counters, columns=sub.columns)

In [1025]:
new_tmp[new_tmp.sum(axis=1)==0]

,Id,Predicted2,Predicted3,Predicted4,Predicted6,Predicted7,Predicted8,Predicted9,Predicted10,Predicted12,...,Predicted559,Predicted562,Predicted563,Predicted564,Predicted565,Predicted566,Predicted567,Predicted568,Predicted570,Predicted571
652,8caed2aa-21bc-11ea-a13a-137349068a90,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2763,878e5d36-21bc-11ea-a13a-137349068a90,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3331,972bfece-21bc-11ea-a13a-137349068a90,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3595,9060bc92-21bc-11ea-a13a-137349068a90,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4150,95199e0c-21bc-11ea-a13a-137349068a90,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [1026]:
new_tmp.sum(axis=1).sum()

5669

In [1027]:
new_tmp.to_csv("subnew.csv", index=False)

In [1028]:
print(len(counters), len(pred), len(new_tmp)) # check for number of images

4746 4746 4746


In [1029]:
print(counters[0])

['8a0a8558-21bc-11ea-a13a-137349068a90', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [1030]:
sub_tmp = pd.DataFrame(counters, columns=sub.columns)

In [1031]:
print(len(sub_tmp))
sub_tmp.head()

4746


,Id,Predicted2,Predicted3,Predicted4,Predicted6,Predicted7,Predicted8,Predicted9,Predicted10,Predicted12,...,Predicted559,Predicted562,Predicted563,Predicted564,Predicted565,Predicted566,Predicted567,Predicted568,Predicted570,Predicted571
0,8a0a8558-21bc-11ea-a13a-137349068a90,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8a00c13a-21bc-11ea-a13a-137349068a90,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8a0c311e-21bc-11ea-a13a-137349068a90,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,8a0cb436-21bc-11ea-a13a-137349068a90,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8a1a9696-21bc-11ea-a13a-137349068a90,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [1032]:
sub_tmp["Id"][0]

'8a0a8558-21bc-11ea-a13a-137349068a90'

In [1033]:
sub_tmp.to_csv("./sub_tmp.csv", index=False)

In [1034]:
df_test_info = load_testinfo()
df_test_info.head()

,image_id,seq_id
0,8b31d3be-21bc-11ea-a13a-137349068a90,a91ebc18-0cd3-11eb-bed1-0242ac1c0002
1,8cf202be-21bc-11ea-a13a-137349068a90,a91ebc18-0cd3-11eb-bed1-0242ac1c0002
2,8a87e62e-21bc-11ea-a13a-137349068a90,a91ebc18-0cd3-11eb-bed1-0242ac1c0002
3,8e6994f4-21bc-11ea-a13a-137349068a90,a91ebc18-0cd3-11eb-bed1-0242ac1c0002
4,948b29e2-21bc-11ea-a13a-137349068a90,a91ebc18-0cd3-11eb-bed1-0242ac1c0002


In [1036]:
unique_countries = df_test_info_gps.CC.unique()
print(unique_countries)
# train countries are ['EC' 'RW' 'KE' 'BR' 'BZ' 'ZZ' 'GT' 'LA']
df_test_info_gps.CC.value_counts()

['BZ' 'KE' 'ZZ' 'RW' 'BR' 'EC']


BZ    23910
ZZ    18403
KE    15801
RW     1959
BR       95
EC       46
Name: CC, dtype: int64

In [1037]:
sub_merged = sub_tmp.merge(df_test_info, left_on="Id", right_on="image_id", how="right")

In [1039]:
print(len(df_test_info))
print(len(sub_merged))
print(len(sub_tmp))
sub_merged[sub_merged.sum(axis=1) < 1].shape

60214
60214
4746


(55473, 207)

In [1038]:
##sum_counts = []
##for i in range(len(sub_merged)):
##    sum_counts.append(sum(sub_merged.iloc[i][col_Predicted]))

In [1040]:
##sub_tmp["total"] =  sum_counts
## count the mnumber of animals in this image
#sub_merged[sub_merged["Id"] == "8a4cef9c-21bc-11ea-a13a-137349068a90"][col_Predicted].sum(axis=1)
sub_merged[sub_merged["seq_id"] == "2966a68a-6fe2-11eb-844f-0242ac1c0002"].fillna(0)



,Id,Predicted2,Predicted3,Predicted4,Predicted6,Predicted7,Predicted8,Predicted9,Predicted10,Predicted12,...,Predicted563,Predicted564,Predicted565,Predicted566,Predicted567,Predicted568,Predicted570,Predicted571,image_id,seq_id
50391,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8abfc332-21bc-11ea-a13a-137349068a90,2966a68a-6fe2-11eb-844f-0242ac1c0002
50392,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,94df948c-21bc-11ea-a13a-137349068a90,2966a68a-6fe2-11eb-844f-0242ac1c0002


In [1041]:
sub_merged.head()

,Id,Predicted2,Predicted3,Predicted4,Predicted6,Predicted7,Predicted8,Predicted9,Predicted10,Predicted12,...,Predicted563,Predicted564,Predicted565,Predicted566,Predicted567,Predicted568,Predicted570,Predicted571,image_id,seq_id
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8b31d3be-21bc-11ea-a13a-137349068a90,a91ebc18-0cd3-11eb-bed1-0242ac1c0002
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8cf202be-21bc-11ea-a13a-137349068a90,a91ebc18-0cd3-11eb-bed1-0242ac1c0002
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8a87e62e-21bc-11ea-a13a-137349068a90,a91ebc18-0cd3-11eb-bed1-0242ac1c0002
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8e6994f4-21bc-11ea-a13a-137349068a90,a91ebc18-0cd3-11eb-bed1-0242ac1c0002
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,948b29e2-21bc-11ea-a13a-137349068a90,a91ebc18-0cd3-11eb-bed1-0242ac1c0002


In [713]:
sub_mean = sub_merged.groupby("seq_id").mean().fillna(0)
sub_std = sub_merged.groupby("seq_id").var().fillna(0)

#sub_median = sub_merged.groupby("seq_id").median().fillna(0)
#sub_mode = sub_merged.groupby("seq_id").agg(lambda x: scipy.stats.mode(x)[0][0]).fillna(0)

In [932]:
def func(group):
    return group.max() + group.std()

In [978]:
def func2(group):
    return group.max() + group.mad()

In [1042]:
sub_max = sub_merged.groupby("seq_id").max(numeric_only=True).fillna(0)

In [935]:
sub_maxg = sub_merged.groupby("seq_id").agg(func).fillna(0)

In [979]:
sub_maxmad = sub_merged.groupby("seq_id").agg(func2).fillna(0)

In [59]:
sub_mode = sub_mode.drop(["Id","id"], axis=1)

KeyError: "['id'] not found in axis"

In [1043]:
sub_result = sub_max

In [1044]:
print(sub_result[sub_result.isnull().sum(axis=1) > 0])

Empty DataFrame
Columns: [Predicted2, Predicted3, Predicted4, Predicted6, Predicted7, Predicted8, Predicted9, Predicted10, Predicted12, Predicted13, Predicted14, Predicted15, Predicted16, Predicted20, Predicted24, Predicted25, Predicted26, Predicted32, Predicted44, Predicted50, Predicted62, Predicted67, Predicted70, Predicted71, Predicted72, Predicted73, Predicted74, Predicted77, Predicted78, Predicted80, Predicted83, Predicted86, Predicted89, Predicted90, Predicted91, Predicted92, Predicted94, Predicted96, Predicted97, Predicted98, Predicted99, Predicted100, Predicted101, Predicted102, Predicted103, Predicted104, Predicted106, Predicted108, Predicted110, Predicted111, Predicted112, Predicted113, Predicted114, Predicted115, Predicted116, Predicted118, Predicted119, Predicted120, Predicted121, Predicted122, Predicted123, Predicted124, Predicted127, Predicted129, Predicted130, Predicted133, Predicted134, Predicted139, Predicted141, Predicted142, Predicted144, Predicted145, Predicted147, 

In [1045]:
sub_result[sub_result.sum(axis=1) == 0].shape

(9543, 204)

In [1046]:
sub_result.head()

,Predicted2,Predicted3,Predicted4,Predicted6,Predicted7,Predicted8,Predicted9,Predicted10,Predicted12,Predicted13,...,Predicted559,Predicted562,Predicted563,Predicted564,Predicted565,Predicted566,Predicted567,Predicted568,Predicted570,Predicted571
seq_id,,,,,,,,,,,,,,,,,,,,,
2966a68a-6fe2-11eb-844f-0242ac1c0002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
296781a4-6fe2-11eb-844f-0242ac1c0002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2967a1fc-6fe2-11eb-844f-0242ac1c0002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2967bdd6-6fe2-11eb-844f-0242ac1c0002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29681182-6fe2-11eb-844f-0242ac1c0002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [1047]:
#sub = sub[["Id_x"] + col_Predicted].rename(columns={"Id_x": "Id"})
sub_result.reset_index()
#sub_mean = sub_mean.round(0).astype(int)
sub_result = sub_result.rename(columns={"seq_id": "Id"})
sub_result.index.rename('Id', inplace=True)

sub_result.to_csv("sub.csv", index=True)


In [985]:

sub_result.head()

,Predicted2,Predicted3,Predicted4,Predicted6,Predicted7,Predicted8,Predicted9,Predicted10,Predicted12,Predicted13,...,Predicted559,Predicted562,Predicted563,Predicted564,Predicted565,Predicted566,Predicted567,Predicted568,Predicted570,Predicted571
Id,,,,,,,,,,,,,,,,,,,,,
2966a68a-6fe2-11eb-844f-0242ac1c0002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
296781a4-6fe2-11eb-844f-0242ac1c0002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2967a1fc-6fe2-11eb-844f-0242ac1c0002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2967bdd6-6fe2-11eb-844f-0242ac1c0002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29681182-6fe2-11eb-844f-0242ac1c0002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [986]:
sub_result.describe()

,Predicted2,Predicted3,Predicted4,Predicted6,Predicted7,Predicted8,Predicted9,Predicted10,Predicted12,Predicted13,...,Predicted559,Predicted562,Predicted563,Predicted564,Predicted565,Predicted566,Predicted567,Predicted568,Predicted570,Predicted571
count,11057.000000,11057.000000,11057.000000,11057.000000,11057.0,11057.000000,11057.000000,11057.000000,11057.0,11057.0,...,11057.000000,11057.0,11057.0,11057.00000,11057.00000,11057.000000,11057.0,11057.000000,11057.0,11057.0
mean,0.005002,0.009664,0.000904,0.009723,0.0,0.008910,0.000362,0.032326,0.0,0.0,...,0.000481,0.0,0.0,0.00009,0.00009,0.000181,0.0,0.000543,0.0,0.0
std,0.138161,0.102439,0.030061,0.107341,0.0,0.127233,0.019017,0.180276,0.0,0.0,...,0.027531,0.0,0.0,0.00951,0.00951,0.013449,0.0,0.023289,0.0,0.0
min,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.00000,0.00000,0.000000,0.0,0.000000,0.0,0.0
25%,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.00000,0.00000,0.000000,0.0,0.000000,0.0,0.0
50%,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.00000,0.00000,0.000000,0.0,0.000000,0.0,0.0
75%,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.00000,0.00000,0.000000,0.0,0.000000,0.0,0.0
max,7.200000,2.375000,1.000000,2.500000,0.0,3.720000,1.000000,2.500000,0.0,0.0,...,2.320000,0.0,0.0,1.00000,1.00000,1.000000,0.0,1.000000,0.0,0.0


In [987]:
sub_result[col_Predicted].sum(axis=0).sum()

5302.594641912321

In [892]:
hard_ones = [257, 130, 133, 402, 406, 407, 408, 412, 415, 417, 420, 296, 306, 571, 322, 323, 330, 77, 333, 80, 339, 342, 92, 349, 355, 356, 229, 114, 119, 250, 251, 252, 253]


In [893]:
hard_ones = sorted(hard_ones)

In [894]:
hard_ids = ["Predicted" + str(h) for h in hard_ones]
class_map.name(hard_ones[0])

AttributeError: 'ClassMapping' object has no attribute 'name'

In [ ]:
sub_result[hard_ids].sum(axis=0).sum()